In [7]:
import sys
import os
import pandas as pd
import matplotlib.pyplot as plt
# Remplacez 'chemin/vers/votre/projet/module' par le chemin absolu réel vers le dossier "module" via la commande pwd
module_path = '/Users/eliotsouthon/Desktop/POC/Code/Modules'  
sys.path.append(module_path)
import ma_bibliotheque 

In [8]:
csv_file_paths=ma_bibliotheque.path_data('/Users/eliotsouthon/Desktop/POC/Data/Data France/Data_silver_france')
# Affiche la liste des chemins pour vérifier
print("Liste des chemins des fichiers CSV :")
for path in csv_file_paths:
    print(path)

Liste des chemins des fichiers CSV :
/Users/eliotsouthon/Desktop/POC/Data/Data France/Data_silver_france/Data_electrcite/energy_1_76.csv
/Users/eliotsouthon/Desktop/POC/Data/Data France/Data_silver_france/Data_electrcite/energy_1_75.csv
/Users/eliotsouthon/Desktop/POC/Data/Data France/Data_silver_france/Data_electrcite/energy_1_11.csv
/Users/eliotsouthon/Desktop/POC/Data/Data France/Data_silver_france/Data_electrcite/energy_1_28.csv
/Users/eliotsouthon/Desktop/POC/Data/Data France/Data_silver_france/Data_electrcite/energy_1_32.csv
/Users/eliotsouthon/Desktop/POC/Data/Data France/Data_silver_france/Data_electrcite/energy_1_27.csv
/Users/eliotsouthon/Desktop/POC/Data/Data France/Data_silver_france/Data_electrcite/energy_1_24.csv
/Users/eliotsouthon/Desktop/POC/Data/Data France/Data_silver_france/Data_electrcite/energy_1_93.csv
/Users/eliotsouthon/Desktop/POC/Data/Data France/Data_silver_france/Data_electrcite/energy_1_44.csv
/Users/eliotsouthon/Desktop/POC/Data/Data France/Data_silver_fr

In [10]:
# Définir les chemins vers les fichiers
energy_file = "/Users/eliotsouthon/Desktop/POC/Data/Data France/Data_silver_france/Data_electrcite/energy_1_75.csv"
weather_file = "/Users/eliotsouthon/Desktop/POC/Data/Data France/Data_silver_france/Data_weather/weather_1_75.csv"

In [11]:
# Fonction unique

def preprocess_energy_weather(energy_path, weather_path, sep=';', encoding='utf-8', rolling_window=3):
    """
    Préprocesser les fichiers CSV pour les données énergétiques et météorologiques :
    - Convertir la colonne 'Date' en format datetime.
    - Fusionner les deux DataFrames sur la colonne 'Date'.
    - Imputer les valeurs manquantes avec une moyenne glissante sur les colonnes numériques.
    - Supprimer les lignes restantes contenant des valeurs manquantes.

    :param energy_path: Chemin du fichier CSV des données énergétiques.
    :param weather_path: Chemin du fichier CSV des données météorologiques.
    :param sep: Séparateur des colonnes dans les fichiers CSV.
    :param encoding: Encodage des fichiers CSV.
    :param rolling_window: Taille de la fenêtre glissante pour l'imputation.
    :return: DataFrame fusionné et nettoyé.
    """
    # Charger les données
    df_energy = pd.read_csv(energy_path, sep=sep, encoding=encoding)
    df_weather = pd.read_csv(weather_path, sep=sep, encoding=encoding)
    
    # Convertir les colonnes 'Date' en datetime
    df_energy['Date'] = pd.to_datetime(df_energy['Date'])
    df_weather['Date'] = pd.to_datetime(df_weather['Date'])
    
    # Fusionner les deux DataFrames sur 'Date'
    df_fusion = pd.merge(df_energy, df_weather, on="Date", how="inner")
    
    # Identifier les colonnes numériques pour l'imputation
    numeric_columns = df_fusion.select_dtypes(include=['number']).columns
    
    # Imputer les valeurs manquantes avec une moyenne glissante
    for col in numeric_columns:
        df_fusion[col] = df_fusion[col].fillna(
            df_fusion[col].rolling(window=rolling_window, min_periods=1).mean()
        )
    
    # Supprimer les lignes contenant des valeurs manquantes
    df_fusion = df_fusion.dropna()
    
    return df_fusion


In [ ]:
# Appeler la fonction
df_cleaned = preprocess_energy_weather(energy_file, weather_file)

In [13]:
df_cleaned.head()

,Date,Consommation brute électricité (MW),Temperature,Humidity,Wind Force
1,2013-01-01 00:00:00+00:00,5468.0,8.0,88.0,5.5
2,2013-01-01 00:00:00+00:00,5468.0,8.3,88.0,6.1
3,2013-01-01 00:00:00+00:00,5468.0,8.0,95.0,5.8
4,2013-01-01 01:00:00+00:00,5479.5,6.8,91.5,10.0
5,2013-01-01 01:00:00+00:00,5479.5,9.0,82.0,3.2


In [14]:
ma_bibliotheque.check_completeness(df_cleaned)

Analyse des valeurs manquantes par colonne :
 Date                                   0
Consommation brute électricité (MW)    0
Temperature                            0
Humidity                               0
Wind Force                             0
dtype: int64

Nombre total de lignes: 30672

Lignes contenant des valeurs manquantes: 0

Le DataFrame est complet (aucune valeur manquante).


In [16]:
numero_region=75
numero_pays=ma_bibliotheque.get_country_iso_numeric("france")

# Construire le nom du fichier en utilisant la première valeur de la colonne 'Code INSEE région'
nom_fichier = f"e_w_{numero_pays}_{numero_region}.csv"

# Définir le chemin complet en joignant le chemin et le nom du fichier
chemin_complet = f"/Users/eliotsouthon/Desktop/POC/Data/Data France/Data_silver_france/{nom_fichier}"

# Sauvegarder le DataFrame en CSV
df_cleaned.to_csv(chemin_complet, index=False, sep=';', encoding='utf-8')